## Test with ResNet in pytorch

In [1]:
from trainer import Trainer
from resnet import fineTuneResNet50,lessFilterResNet50, resnet50
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_json('Data/processed_train.json')
print(train_df.shape)

(1604, 7)


In [3]:
print(np.array(train_df.ix[0, 'norm_band_mixed']).shape)

(75, 75, 3)


C:\Users\khyeh\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [4]:
#model, unused_dict = modResNet18(pretrained=True, skip_headers=['bn2', 'fc'])

In [5]:
#print(unused_dict.keys())

In [6]:
from sklearn.model_selection import KFold
fold_num = 5

In [7]:
import torch

## Cross Validation

## cross validate fine-tune @ fc layers

In [8]:
kf = KFold(n_splits=fold_num, shuffle=True, random_state=519)
counter  = 0
mean_err = 0

for train_index, valid_index in kf.split(train_df):
     
    is_transfer_learning = True
    model, to_train_dict = fineTuneResNet50(pretrained=True)

    print("Dict. to train:")
    print(to_train_dict.keys())
    
    # set only the parameters in to_train_dict to train
    # others: requires_grad = False
    to_train_parmas = []
    for name, param in model.named_parameters():
        if name in to_train_dict.keys():
            print('\nTrain params:', name)
            to_train_parmas.append(param)
        else:
            print(name, end=" ")
            param.requires_grad = False
    
    optimizer = torch.optim.Adam(to_train_parmas, lr=0.001, weight_decay=0.001)
    
    '''
    is_transfer_learning = False
    model = lessFilterResNet50()
    # testing snap shot for essemble
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)
    '''
    tr = Trainer(
            model,
            optimizer,
            epochs=1000,
            milestones=[50, 600],
            gamma=0.1,
            batch_size=128, 
            use_cuda=True, 
            gpu_idx=0)
    train = train_df.loc[train_index].reset_index(drop=True)
    valid = train_df.loc[valid_index].reset_index(drop=True)

    tr.train(train, valid_df=valid, is_transfer_learning=is_transfer_learning)
    
    counter += 1
    tr.save('Trained_model/fine_tune_resent50_' + str(counter) + '.db')
    del model, tr, optimizer



==== Pre-trained layers ==== 


conv1.weight
bn1.running_mean
bn1.running_var
bn1.weight
bn1.bias
layer1.0.conv1.weight
layer1.0.bn1.running_mean
layer1.0.bn1.running_var
layer1.0.bn1.weight
layer1.0.bn1.bias
layer1.0.conv2.weight
layer1.0.bn2.running_mean
layer1.0.bn2.running_var
layer1.0.bn2.weight
layer1.0.bn2.bias
layer1.0.conv3.weight
layer1.0.bn3.running_mean
layer1.0.bn3.running_var
layer1.0.bn3.weight
layer1.0.bn3.bias
layer1.0.downsample.0.weight
layer1.0.downsample.1.running_mean
layer1.0.downsample.1.running_var
layer1.0.downsample.1.weight
layer1.0.downsample.1.bias
layer1.1.conv1.weight
layer1.1.bn1.running_mean
layer1.1.bn1.running_var
layer1.1.bn1.weight
layer1.1.bn1.bias
layer1.1.conv2.weight
layer1.1.bn2.running_mean
layer1.1.bn2.running_var
layer1.1.bn2.weight
layer1.1.bn2.bias
layer1.1.conv3.weight
layer1.1.bn3.running_mean
layer1.1.bn3.running_var
layer1.1.bn3.weight
layer1.1.bn3.bias
layer1.2.conv1.weight
layer1.2.bn1.running_mean
layer1.2.bn1.running_var
layer1.

epoch= 10: Train set: Average loss: 0.5923, Accuracy: 924/1283 (72.02%)
Valid set: Average loss: 0.5505, Accuracy: 229/321 (71.34%)
epoch= 20: Train set: Average loss: 0.5006, Accuracy: 955/1283 (74.43%)
Valid set: Average loss: 0.5067, Accuracy: 234/321 (72.90%)
epoch= 30: Train set: Average loss: 0.5622, Accuracy: 933/1283 (72.72%)
Valid set: Average loss: 0.5028, Accuracy: 230/321 (71.65%)
epoch= 40: Train set: Average loss: 0.4924, Accuracy: 957/1283 (74.59%)
Valid set: Average loss: 0.4937, Accuracy: 230/321 (71.65%)
epoch= 50: Train set: Average loss: 0.5002, Accuracy: 933/1283 (72.72%)
Valid set: Average loss: 0.5134, Accuracy: 235/321 (73.21%)


KeyboardInterrupt: 

## cross validate less filter ResNet5

In [ ]:
kf = KFold(n_splits=fold_num, shuffle=True, random_state=519)
counter  = 0
mean_err = 0

for train_index, valid_index in kf.split(train_df):
     
    
    is_transfer_learning = False
    model = lessFilterResNet50()
    # testing snap shot for essemble
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)
    
    tr = Trainer(
            model,
            optimizer,
            epochs=1000,
            milestones=[50, 600],
            gamma=0.1,
            batch_size=128, 
            use_cuda=True, 
            gpu_idx=0)
    train = train_df.loc[train_index].reset_index(drop=True)
    valid = train_df.loc[valid_index].reset_index(drop=True)

    tr.train(train, valid_df=valid, is_transfer_learning=is_transfer_learning)
    
    counter += 1
    tr.save('Trained_model/fine_tune_resent50_' + str(counter) + '.db')
    del model, tr, optimizer